# Data Preprocessing and Feature Engineering

## Table of Contents

- Load Data
- Data Exploration
    - Data Types
    - Missing Data
    - Variable Content
- Data Preprocessing
- Feature Engineering
    - Loan Related
    - User Related 
- Good-Bad Loans
- Train-Test Split
- Feature Engineering
    - User Repayment Related

In [1]:
from contextlib import closing
import sqlite3
from copy import deepcopy
import numpy as np
import pandas as pd

# Utils
from sklearn.model_selection import train_test_split

# Load Data

In [2]:
with closing(sqlite3.connect("../data/raw/database.db")) as conn:
    df_loans = pd.read_sql_query("SELECT * FROM loans", conn)
    df_loan_repayments = pd.read_sql_query("SELECT * FROM loan_repayments", conn)
    df_transactions = pd.read_sql_query("SELECT * FROM transactions", conn)

In [3]:
print(f"number of instances: {df_loans.shape[0]:,}")

df_loans.head(5)

number of instances: 6,746


,id,user_id,amount,total_amount,due_amount,due_date,status,created_at
0,0,3070,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:47:29.575000+00:00
1,1,2546,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 00:49:51.763000+00:00
2,2,2413,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 01:24:40.537000+00:00
3,3,2585,6000.0,6045.28,6459000000,2022-05-02,debt_collection,2022-02-01 02:52:59.803000+00:00
4,4,2556,6000.0,6045.28,6459000000,2022-05-02,repaid,2022-02-01 02:53:07.123000+00:00


In [4]:
print(f"number of instances: {df_loan_repayments.shape[0]:,}")

df_loan_repayments.head(5)

number of instances: 172,445


,id,loan_id,type,amount,status,created_at
0,1,2,autopilot,269.7,paid,2022-02-01 00:10:08.057000+00:00
1,2,4,autopilot,2550.0,paid,2022-02-01 00:10:08.102000+00:00
2,3,53,pix,1500.0,defaulted,2022-02-01 13:09:53.757000+00:00
3,4,22,autopilot,630.0,paid,2022-02-02 01:21:34.278000+00:00
4,5,70,autopilot,120.0,paid,2022-02-02 01:21:34.449000+00:00


In [5]:
print(f"number of instances: {df_transactions.shape[0]:,}")

df_transactions.head(5)

number of instances: 1,448,684


,id,user_id,amount,status,capture_method,payment_method,installments,card_brand,created_at
0,1,2546,449.6,approved,emv,credit,1,mastercard,2021-12-02 13:41:53.548000+00:00
1,2,2546,269.8,approved,emv,credit,2,mastercard,2021-12-02 15:57:58.742000+00:00
2,3,2546,149.9,approved,contactless,debit,1,visa,2021-12-02 19:36:31.859000+00:00
3,4,2546,142.0,approved,contactless,credit,1,visa,2021-12-02 19:37:11.442000+00:00
4,5,2546,156.0,approved,emv,debit,1,mastercard,2021-12-02 20:20:26.648000+00:00


# Data Exploration

- Data Types
- Missing Data
- Variable Content

## Data Types

In [6]:
print(f"{df_loans.dtypes}\n")

df_loans["due_date"] = pd.to_datetime(df_loans["due_date"])
df_loans["created_at"] = pd.to_datetime(pd.to_datetime(df_loans["created_at"], format='ISO8601').dt.strftime('%Y-%m-%d'))

print(df_loans.dtypes)

id                int64
user_id           int64
amount          float64
total_amount    float64
due_amount        int64
due_date         object
status           object
created_at       object
dtype: object

id                       int64
user_id                  int64
amount                 float64
total_amount           float64
due_amount               int64
due_date        datetime64[ns]
status                  object
created_at      datetime64[ns]
dtype: object


In [7]:
print(f"{df_loan_repayments.dtypes}\n")

df_loan_repayments["created_at"] = pd.to_datetime(pd.to_datetime(df_loan_repayments["created_at"], format='ISO8601').dt.strftime('%Y-%m-%d'))

print(df_loan_repayments.dtypes)

id              int64
loan_id         int64
type           object
amount        float64
status         object
created_at     object
dtype: object

id                     int64
loan_id                int64
type                  object
amount               float64
status                object
created_at    datetime64[ns]
dtype: object


In [8]:
print(f"{df_transactions.dtypes}\n")

df_transactions["created_at"] = pd.to_datetime(pd.to_datetime(df_transactions["created_at"], format='ISO8601').dt.strftime('%Y-%m-%d'))

print(df_transactions.dtypes)

id                  int64
user_id             int64
amount            float64
status             object
capture_method     object
payment_method     object
installments        int64
card_brand         object
created_at         object
dtype: object

id                         int64
user_id                    int64
amount                   float64
status                    object
capture_method            object
payment_method            object
installments               int64
card_brand                object
created_at        datetime64[ns]
dtype: object


## Missing Data

In [9]:
df_loans.isnull().sum()

id              0
user_id         0
amount          0
total_amount    0
due_amount      0
due_date        0
status          0
created_at      0
dtype: int64

In [10]:
df_loan_repayments.isnull().sum()

id            0
loan_id       0
type          0
amount        0
status        0
created_at    0
dtype: int64

In [11]:
df_transactions.isnull().sum()

id                0
user_id           0
amount            0
status            0
capture_method    0
payment_method    0
installments      0
card_brand        0
created_at        0
dtype: int64

## Variable Content

In [12]:
print(f"number of loans: {df_loans['id'].nunique():,}")
print(f"number of users: {df_loans['user_id'].nunique():,}")
print(f"loans originating (years): {df_loans['created_at'].dt.year.unique()}")

number of loans: 6,746
number of users: 3,154
loans originating (years): [2022]


In [13]:
print(f"number of loans being repayed: {df_loan_repayments['loan_id'].nunique():,}")
print(f"repayments originating (years): {df_loan_repayments['created_at'].dt.year.unique()}")

number of loans being repayed: 6,598
repayments originating (years): [2022 2023]


In [14]:
print(f"number of user: {df_transactions['user_id'].nunique():,}")
print(f"transactions originating (years): {df_transactions['created_at'].dt.year.unique()}")

number of user: 3,150
transactions originating (years): [2021 2022 2023]


**Notes:**

- `table_transactions` is missings information about 4 users who have taken a loan
- `table_loan_repayments` is missing information about 148 loans

In [15]:
df_transactions["capture_method"].unique()

array(['emv', 'contactless', 'payment_link', 'ecommerce', 'mpos',
       'payment_link_web'], dtype=object)

**Notes:**

- emv: Europay, Mastercard and Visa
- mpos: Mobile Point of Sale

In [16]:
df_transactions["card_brand"].unique()

array(['mastercard', 'visa', 'elo', 'hipercard', 'amex'], dtype=object)

**Notes:**

- Elo and Hipercard are specific to Brazil

# Data Preprocessing 

- **Handle invalid data**
    - exclude loans that were cancelled or created by error both from `table_loans` and `table_loan_repayments`
    - `table_loan_repayments` is missing information about 148 loans, thus cannot derive information of whether it is a good/bad loan
        - remove these loans from `table_loans`
        - check if there are inconcistences with the users, and if yes repeat the previous step
    - due_amount in table_loans needs to be converted to a float amount
- **Misc**
    - auxiliary repayment status feature, where `refunded` is merged with `paid` (refunded: a loan repayment that happened but was fully refunded to the user)
    - auxiliary loan status features, where `debt_collection` and `debt_repaid` are merged together as `debt` 
- **Notes**
    - `table_transactions` is missing information about 4 users who have taken a loan
        - keep and consider the cold start problem (unavailable purchasing and lending behavior)

In [17]:
exclude = ["error", "cancelled"]
excluded_ids = set(df_loans[df_loans["status"].isin(exclude)]["id"])

# exclude loans that were cancelled or created by error
df_loans = df_loans[~df_loans["id"].isin(excluded_ids)]
df_loan_repayments = df_loan_repayments[~df_loan_repayments["loan_id"].isin(excluded_ids)]

In [18]:
# table_loan_repayments is missing information about some loans
# remove these loans from table_loans
missing_loans = set(df_loans['id']) - set(df_loan_repayments['loan_id'])

df_loans = df_loans[~df_loans['id'].isin(missing_loans)]

In [19]:
# table_transactions contains information about users who aren't present in table_loans
# remove these users from table_transactions
missing_users = set(df_transactions['user_id']) - set(df_loans['user_id'])

df_transactions = df_transactions[~df_transactions['user_id'].isin(missing_users)]

In [20]:
print(f"number of loans: {df_loans['id'].nunique():,}")
print(f"number of users: {df_loans['user_id'].nunique():,}")
print(f"loans originating (years): {df_loans['created_at'].dt.year.unique()}\n")

print(f"number of loans being repayed: {df_loan_repayments['loan_id'].nunique():,}")
print(f"repayments originating (years): {df_loan_repayments['created_at'].dt.year.unique()}\n")

print(f"number of user: {df_transactions['user_id'].nunique():,}")
print(f"transactions originating (years): {df_transactions['created_at'].dt.year.unique()}")

number of loans: 6,588
number of users: 3,046
loans originating (years): [2022]

number of loans being repayed: 6,588
repayments originating (years): [2022 2023]

number of user: 3,043
transactions originating (years): [2021 2022 2023]


In [21]:
# due_amount converted to a float amount
df_loans["due_amount"] = df_loans["due_amount"] / 1_000_000

In [22]:
# merge debt_collection and debt_repaid to debt
df_loans["status_cleaned"] = df_loans["status"].replace({"debt_collection": "debt", "debt_repaid": "debt"})

In [23]:
# merge refunded with paid, since refunded is a repayment that happened but was fully refunded to the user
df_loan_repayments["status_cleaned"] = df_loan_repayments["status"].replace({"refunded": "paid"})

## ✅ checkpoint

In [24]:
df_loans.to_pickle("../data/processed/df_loans.pkl")
df_loan_repayments.to_pickle("../data/processed/df_loan_repayments.pkl")
df_transactions.to_pickle("../data/processed/df_transactions.pkl")

# Feature Engineering

- **Loan Related**
    - *loan term:* length of time over which a loan is to be repaid (days)
    - *loan fees:* charges that have incurred when the loan was taken
    - *loan amount bins* & *loan due amount bins* & *loan fees bins*
    - *loan amount sizes* & *loan due amount sizes*
    - *interest rate* & *interest rate bins*
    - *default ratio:* number of defaulted repayments to total number repayments
    - *median repayment amount*
    - *number of repayments*
    - *ratio of repaid amount to total amount*
    - *max repayment delay:*  maximum number of days taken for a loan to be repayed after due date has been exceeded
    - *due date exceeded*
    - *number of late repayments*
    - *median repayment lag*
- **User Related**
    - *recency:* time since the user's last transaction
    - *frequency:* how often users make a purchase
    - *monetary:* total amount spent on purchases
    - *median number of installments*
    - *median transaction lag:* median time between transactions per user
    - *denied rate:* number of denied transactions to total number of transactions
    - *transactions with installments ratio:* proportion of transactions with installments
    - *credit to debit ratio:* number of credit transactions to number of debit transactions
    - *online to in-person purchases ratio:* number of online purchases to number of in-person purchases
    - *card brand preference:* determine whether an International or Brazil-based card provider is prefered    

## Loan Related

In [25]:
# Loan Term: difference between due_date and created_at
# longer loan terms might correlate with higher risk of default.
df_loans["loan_term"] = (df_loans["due_date"] - df_loans["created_at"]).dt.days

In [26]:
# loan amount bins
for i in range(0, 10_000, 1_000):
    label = f"{int(i/1_000)}k - {int((i+1_000)/1_000)}k"
    df_loans.loc[df_loans["amount"].between(i, i+1_000), "amount_bin"] = label

# loan amount sizes
df_loans.loc[df_loans["amount_bin"].isin(["1k - 2k", "2k - 3k"]), "amount_cat"] = "small"
df_loans.loc[df_loans["amount_bin"].isin(["3k - 4k", "4k - 5k"]), "amount_cat"] = "medium"
df_loans.loc[df_loans["amount_bin"].isin(["5k - 6k", "6k - 7k"]), "amount_cat"] = "large"

In [27]:
# loan due amount bins
for i in range(0, 15_000, 1_000):
    label = f"{int(i/1_000)}k - {int((i+1_000)/1_000)}k"
    df_loans.loc[df_loans["due_amount"].between(i, i+1_000), "due_amount_bin"] = label

# loan due amount sizes
df_loans.loc[df_loans["due_amount_bin"].isin(["1k - 2k", "2k - 3k"]), "due_amount_cat"] = "small"
df_loans.loc[df_loans["due_amount_bin"].isin(["3k - 4k", "4k - 5k"]), "due_amount_cat"] = "medium"
df_loans.loc[df_loans["due_amount_bin"].isin(["5k - 6k", "6k - 7k", "7k - 8k", "8k - 9k"]), "due_amount_cat"] = "large"

In [28]:
# Loan Fees: difference between total_amount and (net) amount 
# higher fees might correlate with higher risk of default
df_loans["loan_fees"] = df_loans["total_amount"] - df_loans["amount"]

# loan fees bins
for i in range(0, 60, 10):
    label = f"{i} - {i+10}"
    df_loans.loc[df_loans["loan_fees"].between(i, i+10), "loan_fees_bin"] = label

In [29]:
# Interest Rate: based on the formula A=P*(1+r*t) where
# P = Principal Amount
# R = Rate of interest
# t = Number of years
# A = Total accrued amount (both principal and the interest)
df_loans["interest_rate"] = (
    (df_loans["due_amount"] - df_loans["amount"] * 1) / (df_loans["amount"] * df_loans["loan_term"] / 365)  *100
).round()

# interest rate bins
df_loans.loc[df_loans["interest_rate"] < 40, "interest_rate_bin"] = "small"
df_loans.loc[df_loans["interest_rate"].between(40, 100), "interest_rate_bin"] = "medium"
df_loans.loc[df_loans["interest_rate"] > 100, "interest_rate_bin"] = "large"

In [30]:
df_temp = (
    pd.get_dummies(df_loan_repayments['status_cleaned'])
    .rename(columns={"defaulted": "status_defaulted", "paid": "status_paid", "loan_id": "id"})
)
df_loan_repayments = pd.concat([df_loan_repayments, df_temp], axis=1)
df_temp = df_loan_repayments.groupby(["loan_id"], as_index=False).agg(
    {
        "status_defaulted": lambda repayment : repayment.sum(),
        "status_paid": lambda repayment : repayment.sum(),
    }
).rename(columns={"loan_id": "id"})
df_loans = df_loans.merge(df_temp, how="left", on="id")

# Default Ratio: number of defaulted repayments to total number repayments
df_loans["rate_default"] = df_loans["status_defaulted"] / (df_loans["status_defaulted"] + df_loans["status_paid"]) # smoothing

df_loan_repayments = df_loan_repayments.drop(columns=["status_defaulted", "status_paid"])

In [31]:
# Median Repayment Amount
# Number of Repayments
df_temp = df_loan_repayments.groupby(["loan_id"], as_index=False).agg(
    {
        "amount": lambda repayment : repayment.median(),
        "id": lambda repayment : len(repayment),
    }
).rename(columns={"loan_id": "id", "amount": "median_repayment_amount", "id": "num_repayments"})
df_loans = df_loans.merge(df_temp, how="left", on="id")

In [32]:
# auxiliary amount column to account for defaulted and refunded payments
df_loan_repayments["temp_amount"] = 0  # defaulted and refunded repayments should be excluded in sum
df_loan_repayments.loc[df_loan_repayments["status"] == "paid", "temp_amount"] = (
    df_loan_repayments.loc[df_loan_repayments["status"] == "paid", "amount"]
)

# Repaid Amount
df_temp = df_loan_repayments.groupby("loan_id", as_index=False)["temp_amount"].sum().rename(columns={"loan_id": "id", "temp_amount": "repaid_amount"})
df_loans = df_loans.merge(df_temp, how="left", on="id")

# Ratio of Repaid Amount to Loan Amount: can be a good indicator of the borrower's repayment behavior
df_loans["ratio_repaid_total"] = df_loans["repaid_amount"] / df_loans["total_amount"]

df_loans = df_loans.drop(columns=["repaid_amount"])
df_loan_repayments = df_loan_repayments.drop(columns=["temp_amount"])

## Loan Related (Repayment History)

In [33]:
df_temp = (
    df_loans[["id", "created_at", "due_date"]]
    .rename(columns={"id": "loan_id", "created_at": "loan_created_at"})
)
df_loan_repayments = df_loan_repayments.merge(df_temp, how="left", on="loan_id")
df_loan_repayments = df_loan_repayments.sort_values(["loan_id", "created_at"])

In [34]:
# Time Since Due Date: length of time between due date and repayment
df_loan_repayments["days_since_due_date"] = (df_loan_repayments["created_at"] - df_loan_repayments["due_date"]).dt.days
# in case the due date hasn't been exceeded the values will be negative
# default to 0 represent that the due date hasn't been exceeded
df_loan_repayments.loc[df_loan_repayments["days_since_due_date"] < 0, "days_since_due_date"] = 0

# Max Repayment Delay (per loan): maximum number of days taken for a loan to be repayed after due date has been exceeded
df_temp = (
    df_loan_repayments
    .groupby("loan_id", as_index=False)
    ["days_since_due_date"].max()
    .rename(columns={"loan_id": "id", "days_since_due_date": "max_repayment_delay"})
)
df_loans = df_loans.merge(df_temp, how="left", on="id")

In [35]:
# Due Date Exceeded: if due date has been exceeded
df_loan_repayments["due_date_exceeded"] = 0
df_loan_repayments.loc[df_loan_repayments["days_since_due_date"] > 0, "due_date_exceeded"] = 1

df_temp = (
    df_loan_repayments
    .groupby("loan_id", as_index=False)
    ["due_date_exceeded"].max()
    .rename(columns={"loan_id": "id"})
)
df_loans = df_loans.merge(df_temp, how="left", on="id")

In [36]:
# Number of Late Repayments (per loan): this can indicate the borrower's tendency to miss deadlines
df_temp = (
    df_loan_repayments
    .groupby("loan_id", as_index=False)
    ["due_date_exceeded"].sum()
    .rename(columns={"loan_id": "id", "due_date_exceeded": "num_late_repayment"})
)
df_loans = df_loans.merge(df_temp, how="left", on="id")

In [37]:
# Repayment Lag: difference between current repayment date and previous repayment date 
# first repayment date should be compared to the loan creation date
# this can indicate whether the borrower is keeping up with the repayment schedule.
df_loan_repayments["days_lag_repayment"] = df_loan_repayments.groupby(["loan_id"], as_index=False)["created_at"].diff().dt.days
df_loan_repayments.loc[df_loan_repayments["days_lag_repayment"].isna(), "days_lag_repayment"] = (
    df_loan_repayments[df_loan_repayments["days_lag_repayment"].isna()]["created_at"] - \
    df_loan_repayments[df_loan_repayments["days_lag_repayment"].isna()]["loan_created_at"]
).dt.days

# Median Repayment Lag (per loan)
df_temp = (
    df_loan_repayments
    .groupby("loan_id", as_index=False)
    ["days_lag_repayment"].median()
    .apply(np.ceil)
    .rename(columns={"loan_id": "id", "days_lag_repayment": "median_days_lag_repayment"})
)
df_loans = df_loans.merge(df_temp, how="left", on="id")

df_loan_repayments = df_loan_repayments.drop(columns=["loan_created_at", "due_date"])

## User Related

In [38]:
# Recency: Time since the user's last transaction 
# Active users might be more financially active and potentially better candidates for loan repayment

# Frequency: How often user make a purchase
# Users who have a higher frequency of transactions might be more financially active 
# and potentially better candidates for loan repayment

# Monetary: Total amount spent on purchases
# Users who typically make larger transactions or have more consistent transaction amounts 
# might be more likely to repay loans

# to calculte Recency, the current date that will be used for comparison will be the max date of the sample
date_current = df_transactions["created_at"].max()

df_users = df_transactions.groupby("user_id", as_index=False).agg(
    {
        "created_at": lambda date : (date_current - date.max()).days,
        "id": lambda transaction : len(transaction),
        "amount": lambda amount : amount.sum(),
    }
).rename(columns={"created_at": "recency", "id": "frequency", "amount": "monetary"})

In [39]:
# Median Number of Installments
df_temp = df_transactions.groupby("user_id", as_index=False)["installments"].median().rename(columns={"installments": "median_intallments"})
df_users = df_users.merge(df_temp, how="left", on="user_id")

In [40]:
# Median Transaction Lag: median time between transactions per user
df_transactions["days_lag_transaction"] = df_transactions.groupby(["user_id"], as_index=False)["created_at"].diff().dt.days

df_temp = (
    df_transactions.groupby(["user_id"], as_index=False)
    ["days_lag_transaction"].median()
    .rename(columns={"days_lag_transaction": "median_lag_transaction"})
)

df_users = df_users.merge(df_temp, how="left", on="user_id")

In [41]:
df_temp = (
    pd.get_dummies(df_transactions['status'])
    .rename(columns={"approved": "transaction_approved", "denied": "transaction_denied"})
)
df_transactions = pd.concat([df_transactions, df_temp], axis=1)
df_temp = df_transactions.groupby(["user_id"], as_index=False).agg(
    {
        "transaction_approved": lambda transaction : transaction.sum(),
        "transaction_denied": lambda transaction : transaction.sum(),
    }
)
df_users = df_users.merge(df_temp, how="left", on="user_id")

# Denied Ratio: number of denied transactions to total number of transactions 
# Users with a higher denied rate might demonstrate worse financial behavior
df_users["rate_denied"] = df_users["transaction_denied"] / df_users["frequency"]

df_users = df_users.drop(columns=["transaction_approved", "transaction_denied"])
df_transactions = df_transactions.drop(columns=["transaction_approved", "transaction_denied"])

In [42]:
df_transactions.loc[df_transactions["installments"] == 1, "with_installment"] = False
df_transactions.loc[df_transactions["installments"] > 1, "with_installment"] = True

df_temp = df_transactions.groupby(["user_id"], as_index=False)["with_installment"].sum()
df_users = df_users.merge(df_temp, how="left", on="user_id")

# Transactions with Installments Ratio: proportion of transactions with installments
# Users who frequently opt for installment payments might have better financial planning and repayment capabilities
df_users["rate_transactions_installment"] = df_users["with_installment"] / df_users["frequency"]

df_users = df_users.drop(columns=["with_installment"])
df_transactions = df_transactions.drop(columns=["with_installment"])

In [43]:
df_temp = (
    pd.get_dummies(df_transactions['payment_method'])
    .rename(columns={"credit": "payment_method_credit", "debit": "payment_method_debit"})
)
df_transactions = pd.concat([df_transactions, df_temp], axis=1)
df_temp = df_transactions.groupby(["user_id"], as_index=False).agg(
    {
        "payment_method_credit": lambda payment_method : payment_method.sum(),
        "payment_method_debit": lambda payment_method : payment_method.sum(),
    }
)
df_users = df_users.merge(df_temp, how="left", on="user_id")

# Credit to Debit Ratio: number of credit transactions to number of debit transactions 
# Users who rely on credit cards might be more likely to repay loans
df_users["rate_credit_debit"] = (1 + df_users["payment_method_credit"]) / (1 + df_users["payment_method_debit"])  # smoothing

df_users = df_users.drop(columns=["payment_method_credit", "payment_method_debit"])
df_transactions = df_transactions.drop(columns=["payment_method_credit", "payment_method_debit"])

In [44]:
# Based on the capture method, derive whether a purchase was made in-person or online
purchase_in_person, purchase_online = ["emv", "contactless", "mpos"], ["payment_link", "ecommerce", "payment_link_web"]
df_transactions.loc[df_transactions["capture_method"].isin(purchase_in_person), "purchase_type"] = "in_person"
df_transactions.loc[df_transactions["capture_method"].isin(purchase_online), "purchase_type"] = "online"

df_temp = (
    pd.get_dummies(df_transactions['purchase_type'])
    .rename(columns={"in_person": "purchase_in_person", "online": "purchase_online"})
)
df_transactions = pd.concat([df_transactions, df_temp], axis=1)
df_temp = df_transactions.groupby(["user_id"], as_index=False).agg(
    {
        "purchase_in_person": lambda purchase_type : purchase_type.sum(),
        "purchase_online": lambda purchase_type : purchase_type.sum(),
    }
)
df_users = df_users.merge(df_temp, how="left", on="user_id")

# Online to In-Person Purchases Ratio: number of online purchases to number of in-person purchases
# This ratio indicates users' financial habits
df_users["ratio_online_person"] = (1 + df_users["purchase_online"]) / (1 + df_users["purchase_in_person"])  # smoothing

df_users = df_users.drop(columns=["purchase_in_person", "purchase_online"])
df_transactions = df_transactions.drop(columns=["purchase_type", "purchase_in_person", "purchase_online"])

In [45]:
# Based on the card brand, derive whether an International or Brazil-based card provider is being used
card_internation, card_brazil = ["mastercard", "visa", "amex"], ["elo", "hipercard"]
df_transactions.loc[df_transactions["card_brand"].isin(card_internation), "card_type"] = "card_internation"
df_transactions.loc[df_transactions["card_brand"].isin(card_brazil), "card_type"] = "card_brazil"

# Card Brand Preference: determine whether an International or Brazil-based card provider is prefered
df_temp = (
    df_transactions
    .groupby(["user_id"], as_index=False)
    ["card_type"].value_counts()
    .sort_values(['user_id', 'count'], ascending=False)
    .drop_duplicates(['user_id'])
)
df_users = (
    df_users
    .merge(df_temp[["user_id", "card_type"]], how="left", on="user_id")
    .rename(columns={"card_type": "card_preference"})
)

df_transactions = df_transactions.drop(columns=["card_type"])

## ✅ checkpoint

In [46]:
df_loans.to_pickle("../data/processed/df_loans.pkl")
df_users.to_pickle("../data/processed/df_users.pkl")

# Good-Bad Loans

**Definition:**
- To define the goodness of a loan, we need to separate the ones that were paid from the ones that resulted in debt.
- **Loan Debt**
    - Loans resulting in debt are considered bad loans.
- **Paid Loans**
    - The goodness of a paid loan is assessed based on the default rate of that loan benchmarked to the median default rate across all paid loans.
    - However, loans vary, amongst other characteristics, in amount and interest rate, factors that impact the profile of the loan and the user repayment behavior.
    - Thus, to define the goodness of a paid loan we need to benchmark the default rate of a paid loan to the median default rate of the category that it belongs to. The category is defined based on the principal amount (small, medium, large) and the interest rate (small, medium, large).

**Notes:**
- Principal Amount
    - small: R\\$ 1k-3k
    - medium: R\\$ 3k-5k
    - large: R\\$ 5k-7k
- Interest Rate
    - small: < 40%
    - medium: 40% - 100%
    - large: > 100%

In [47]:
df_loans_debt = df_loans[df_loans["status_cleaned"] == "debt"]
df_loans_debt["loan_quality"] = "bad_loan"

df_loans_repaid = df_loans[df_loans["status_cleaned"] == "repaid"]

C:\Users\lampr\AppData\Local\Temp\ipykernel_3284\1799892317.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loans_debt["loan_quality"] = "bad_loan"


In [48]:
# calculate 5-number summary of default rate per loan amount and interest rate combination
stats = df_loans_repaid.groupby(["amount_cat", "interest_rate_bin"], as_index=False)["rate_default"].describe()
stats

,amount_cat,interest_rate_bin,count,mean,std,min,25%,50%,75%,max
0,large,large,857.0,0.043818,0.070628,0.0,0.0,0.021277,0.057692,0.642857
1,large,medium,126.0,0.045592,0.082751,0.0,0.0,0.014709,0.048942,0.400000
2,large,small,3487.0,0.055616,0.087893,0.0,0.0,0.030303,0.071429,0.800000
3,medium,large,48.0,0.064868,0.115035,0.0,0.0,0.035714,0.090909,0.714286
4,medium,medium,11.0,0.064028,0.084505,0.0,0.0,0.046154,0.077853,0.235294
5,medium,small,212.0,0.073365,0.121575,0.0,0.0,0.036700,0.081219,0.769231
6,small,large,56.0,0.063119,0.103677,0.0,0.0,0.000000,0.080103,0.392857
7,small,medium,21.0,0.051298,0.069516,0.0,0.0,0.000000,0.080000,0.266667
8,small,small,229.0,0.102287,0.136684,0.0,0.0,0.045455,0.166667,0.882353


In [49]:
# to assess the quality of a loan, the default rate should meet or exceed the third quartile (Q3) 
# of its corresponding category, determined by loan amount and interest rate
df_loans_repaid = df_loans_repaid.merge(
    (
        stats[["amount_cat", "interest_rate_bin", "75%"]]
        .rename(columns={"75%": "goodness_thres"})
    ), 
    how="left", 
    on=["amount_cat", "interest_rate_bin"]
)
df_loans_repaid["loan_quality"] = (df_loans_repaid["rate_default"] >= df_loans_repaid["goodness_thres"])
df_loans_repaid["loan_quality"] = df_loans_repaid["loan_quality"].replace({True: "bad_loan", False: "good_loan"})

df_loans_repaid = df_loans_repaid.drop(columns=["goodness_thres"])

In [50]:
df_loans = pd.concat([df_loans_debt, df_loans_repaid])

# Train-Test Split

**Note:** 
- We split before calculating the statistics about the repayment behavior of the users, since these need to be based on the training set to avoid data leakage.
- **Existing Users**
    - When evaluating a user on a new loan at timestamp $t$, the repayment behavior of the user is the one observed up until timestamp $t-1$.
- **New Users**
    - When evaluating a user on a new loan at timestamp $t$, the repayment behavior of the user is the one observed up until timestamp $t-1$ of the segment that it belongs to if we have the transaction history of the user, otherwise of the Recent customers segment.

In [51]:
def multi_time_series_split(
    df: pd.DataFrame, 
    primary_key: str, 
    test_size_frac: float, 
    stratify_key: str, 
    random_state: int,
) -> tuple:
    """ Split multiple time-series data into train and test subsets.

    Test set contains the newest records of each time-series.

    Args:
        df:
            Input datataset.
        primary_key:
            Feature that distinguishes the time-series.
        test_size_frac:
            Proportion of the dataset to include in the test split.
        stratify_key:
            Feature to split in a stratified fashion.
        random_state:
            Controls the shuffling applied to the data before applying the split.
    
    Returns:
        Train and test set features and target.
    """
    test_size = round(df.shape[0] * test_size_frac)
    
    df_slice = df.groupby(primary_key, as_index=False).last()

    _, test = train_test_split(df_slice, test_size=test_size, stratify=df_slice[stratify_key], random_state=random_state)

    X_train, X_test = df[~df.index.isin(test.index)], df[df.index.isin(test.index)]
    
    return X_train, X_test

In [52]:
# sort by user and date of loan to get latest loan in test set per user
df_loans = df_loans.sort_values(by=["user_id", "created_at"])

# single_loan flag is used for stratification purposes
# it is important to have in the test set user with only 1 loan because these will be evaluated as new users
# meaning that information related to their lending habits will be erased
df_temp = df_loans.groupby("user_id", as_index=False)["id"].count().rename(columns={"id": "num_loans"})
df_loans = df_loans.merge(df_temp, how="left", on="user_id")
df_loans["single_loan"] = (df_loans["num_loans"] == 1)

df_loans_train, df_loans_test = multi_time_series_split(df_loans, primary_key="user_id", test_size_frac=.2, stratify_key="single_loan", random_state=42)

In [53]:
print(f"Train: number of loans: {df_loans_train['id'].unique().shape[0]:,}\nTest: number of loans: {df_loans_test['id'].unique().shape[0]:,}")

Train: number of loans: 5,270
Test: number of loans: 1,318


In [54]:
single_loan, multi_loan = df_loans_train[df_loans_train["loan_quality"] == "good_loan"].shape[0], df_loans_train[df_loans_train["loan_quality"] == "bad_loan"].shape[0]
print(f"Train:\n\tnumber of good loans: {single_loan:,}\n\tnumber of bad loans: {multi_loan:,}")

single_loan, multi_loan = df_loans_test[df_loans_test["loan_quality"] == "good_loan"].shape[0], df_loans_test[df_loans_test["loan_quality"] == "bad_loan"].shape[0]
print(f"Test:\n\tnumber of good loans: {single_loan:,}\n\tnumber of bad loans: {multi_loan:,}")

Train:
	number of good loans: 3,008
	number of bad loans: 2,262
Test:
	number of good loans: 760
	number of bad loans: 558


In [55]:
single_loan, multi_loan = df_loans_train[df_loans_train["single_loan"] == True].shape[0], df_loans_train[df_loans_train["single_loan"] == False].shape[0]
print(f"Train:\n\tnumber of users with one loan: {single_loan:,}\n\tnumber of users with multiple loans: {multi_loan:,}")

single_loan, multi_loan = df_loans_test[df_loans_test["single_loan"] == True].shape[0], df_loans_test[df_loans_test["single_loan"] == False].shape[0]
print(f"Test:\n\tnumber of users with one loan: {single_loan:,}\n\tnumber of users with multiple loans: {multi_loan:,}")

Train:
	number of users with one loan: 1,188
	number of users with multiple loans: 4,082
Test:
	number of users with one loan: 293
	number of users with multiple loans: 1,025


In [56]:
# drop auxiliary features
df_loans = df_loans.drop(columns=["num_loans", "single_loan"])
df_loans_train, df_loans_test = df_loans_train.drop(columns=["num_loans", "single_loan"]), df_loans_test.drop(columns=["num_loans", "single_loan"])

## ✅ checkpoint

In [57]:
df_loans_train.to_pickle("../data/interim/df_loans_train_raw.pkl")
df_loans_test.to_pickle("../data/interim/df_loans_test_raw.pkl")

# Feature Engineering

**Note:** Statistics about the repayment behavior of the user need to be calculated based on the training set to avoid data leakage. Users not contained in the training set won't have any statistics regarding their repayment behavior (cold start problem).

**User Repayment Related**
- *default ratio:* number of defaulted repayments to total number repayments across all loans
- *median repayment amount:* across all loans
- *maximum repayment delay:* across all loans
- *median repayment lag:* across all loans
- *number of loans*
- *due date exceeded ratio:* number of loans who were repayed outside of deadline to the total number of loans
- *late repayments ratio:* ratio of late repayments to total repayments across all loans
- *manual to automated repayments:* measures the preference of manual repayments over scheduled repayments across all loans
- *repaid loans ratio:* number of fully repaid loans to number of loans

In [58]:
df_temp = df_loans_train.groupby("user_id", as_index=False).agg(
    {
        "status_defaulted": lambda loan : loan.sum(),
        "status_paid": lambda loan : loan.sum(),
    }
)
df_users = df_users.merge(df_temp, how="left", on="user_id")

# Default Ratio: number of defaulted repayments to total number of repayments
# Users with a higher rate demonstrate worse financial behavior
df_users["rate_default"] = df_users["status_defaulted"] / (df_users["status_defaulted"] + df_users["status_paid"])  

df_users = df_users.drop(columns=["status_defaulted", "status_paid"])

In [59]:
# Median Repayment Amount: across all loans
df_temp = df_loans_train.groupby("user_id", as_index=False)['median_repayment_amount'].median()
df_users = df_users.merge(df_temp, how="left", on="user_id")

In [60]:
# Maximum Repayment Delay: across all loans
df_temp = df_loans_train.groupby("user_id", as_index=False)['max_repayment_delay'].max()
df_users = df_users.merge(df_temp, how="left", on="user_id")

In [61]:
# Median Repayment Lag: across all loans
df_temp = df_loans_train.groupby("user_id", as_index=False)['median_days_lag_repayment'].median()
df_users = df_users.merge(df_temp, how="left", on="user_id")

In [62]:
# Number of Loans
# Times Due Date Exceeded: number of loans who were repayed outside of deadline
df_temp = df_loans_train.groupby("user_id", as_index=False).agg(
    {
        "due_date_exceeded": lambda loan : loan.sum(),
        "id": lambda loan : len(loan),
    }
).rename(columns={"id": "num_loans", "due_date_exceeded": "times_due_date_exceeded"})
df_users = df_users.merge(df_temp, how="left", on="user_id")

# Due Data Exceeded Ratio: number of loans who were repayed outside of deadline to the total number of loans
df_users["rate_due_date_exceeded"] = df_users["times_due_date_exceeded"] / df_users["num_loans"]

df_users = df_users.drop(columns=["times_due_date_exceeded"])

In [63]:
# Late Repayments Ratio: ratio of late repayments to total repayments
df_temp = df_loans_train.groupby("user_id", as_index=False).agg(
    {
        "num_repayments": lambda loan : loan.sum(),
        "num_late_repayment": lambda loan : loan.sum(),
    }
).rename(columns={"id": "num_loans", "due_date_exceeded": "times_due_date_exceeded"})
df_users = df_users.merge(df_temp, how="left", on="user_id")
df_users["rate_late_repayment"] = df_users["num_late_repayment"] / df_users["num_repayments"]

df_users = df_users.drop(columns=["num_repayments", "num_late_repayment"])

In [64]:
# Manual to Automated Repayments: measures the preference of manual repayments over scheduled repayments
# Calculated as number times pix was used over autopilot across all loans

# Per loan, calculate the number of repayments made with pix and autopilot
df_temp = (
    pd.get_dummies(df_loan_repayments['type'])
    .rename(columns={"autopilot": "type_autopilot", "pix": "type_pix"})
)
df_loan_repayments = pd.concat([df_loan_repayments, df_temp], axis=1)
df_temp = df_loan_repayments.groupby("loan_id", as_index=False).agg(
    {
        "type_autopilot": lambda payment_type : payment_type.sum(),
        "type_pix": lambda payment_type : payment_type.sum(),
    }
).rename(columns={"loan_id": "id", "type_autopilot": "times_autopilot", "type_pix": "times_pix"})
df_loans_train = df_loans_train.merge(df_temp, how="left", on="id")


# Per user, calculate the number of repayments made with pix and autopilot and calculate the ratio
df_temp = df_loans_train.groupby("user_id", as_index=False).agg(
    {
        "times_autopilot": lambda payment_type : payment_type.sum(),
        "times_pix": lambda payment_type : payment_type.sum(),
    }
)
df_temp["rate_manual_auto_repayments"] = (1 + df_temp["times_pix"]) / (1 + df_temp["times_autopilot"])  # smoothing
df_users = df_users.merge(df_temp[["user_id", "rate_manual_auto_repayments"]], how="left", on="user_id")

df_loan_repayments = df_loan_repayments.drop(columns=["type_autopilot", "type_pix"])
df_loans_train = df_loans_train.drop(columns=["times_autopilot", "times_pix"])

In [65]:
# Loan fully repaid binary indicator 
# If ratio_repaid_total is less than 100% then loan hasn't been fully repaid
df_loans_train["loan_repaied"] = df_loans_train["ratio_repaid_total"].apply(np.floor)

# Repaid Loans Ratio: number of fully repaid loans to number of loans
df_temp = df_loans_train.groupby("user_id", as_index=False).agg(
    {
        "loan_repaied": lambda payment_type : payment_type.sum(),
        "id": lambda payment_type : len(payment_type),
    }
).rename(columns={"id": "num_loans", "loan_repaied": "num_loans_repaied"})
df_temp["rate_repaid_loans"] = df_temp["num_loans_repaied"] / df_temp["num_loans"]
df_users = df_users.merge(df_temp[["user_id", "rate_repaid_loans"]], how="left", on="user_id")

df_loans_train = df_loans_train.drop(columns=["loan_repaied"])

## ✅ checkpoint

In [66]:
df_users.to_pickle("../data/processed/df_users.pkl")